<a href="https://colab.research.google.com/github/Arun-nexus/deep_learning/blob/main/summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df=pd.read_csv(r"C:\Users\Arun\Downloads\train_essays_v1.csv\train_essays_v1.csv")
df

,text,label,prompt_name,source,RDizzl3_seven
0,The Electoral College is a complex system that...,1,Does the electoral college work?,gemini_pro,True
1,"The exploration of Venus, Earth's closest plan...",1,Exploring Venus,gemini_pro,True
2,## Does the Electoral College Work?\n\nThe Ele...,1,Does the electoral college work?,gemini_pro,True
3,"In the vast realm of literature, there exists ...",1,"""A Cowboy Who Rode the Waves""",gemini_pro,True
4,"In the realm of modern transportation, the adv...",1,Driverless cars,gemini_pro,True
...,...,...,...,...,...
3495,"The Electoral College, a distinctive feature o...",1,Does the electoral college work?,gemini_pro,True
3496,The Electoral College is a system of indirect ...,1,Does the electoral college work?,gemini_pro,True
3497,"The electoral College, an integral part of the...",1,Does the electoral college work?,gemini_pro,True
3498,"The Electoral College, a distinctive feature o...",1,Does the electoral college work?,gemini_pro,True


In [ ]:
data=df["text"]
data=data[:2000].to_list()
data

['The Electoral College is a complex system that has been in place since the founding of the United States. It is a system that has both advantages and disadvantages. On the one hand, it ensures that every state has a say in who becomes President, regardless of population. On the other hand, it can lead to situations where a candidate who wins the popular vote does not win the Presidency, as happened in 2016.\n\nThere are a number of arguments in favor of the Electoral College. One argument is that it helps to protect the rights of minority groups. In a direct popular vote, it is possible for a candidate to win by a landslide in a few large states, while losing by narrow margins in many smaller states. This could lead to a situation where the President is elected by a minority of the population. The Electoral College, on the other hand, ensures that every state has a say in the election, regardless of its size.\n\nAnother argument in favor of the Electoral College is that it encourages

In [ ]:
from nltk import word_tokenize

def tokenize(text):
    tokens=[word_tokenize(row.lower()) for row in text]
    return tokens

data_tokens=tokenize(data)

In [ ]:
sequence=[]
for row in data_tokens:
    for word in row:
        sequence.append(word)

In [ ]:
len(sequence)

1147324

In [ ]:
dictionary={"<pad>":0,"<unk>":1}
for vocab in sequence:
    if vocab not in dictionary:
        dictionary[vocab]=len(dictionary)


In [ ]:
vocab_size=len(dictionary)

In [ ]:
def vectorize(text,dicti):
    seq=[]
    for row in text:
        dataset=[dicti.get(word,dicti["<unk>"]) for word in row]
        seq.append(dataset)
    return seq
dataset=vectorize(data_tokens,dictionary)

In [ ]:
dataset[0][-1]

19

In [ ]:
max_len=2000
for row in dataset:
    if len(row)<=max_len:
        row+=[dictionary["<pad>"]]*(max_len-len(row))
    else:
        row=row[:max_len]

In [ ]:
import torch
import torch.nn as nn
import math
from torch.utils.data import DataLoader
tensor_data=torch.tensor(dataset,dtype=torch.long)
loaded_data=DataLoader(tensor_data,batch_size=256)

In [ ]:
class positional_encoding(nn.Module):
    def __init__(self,max_len=100,d_model=512):
        super().__init__()

        pe=torch.zeros(max_len,d_model)
        position=torch.arange(0,max_len).unsqueeze(1)
        div_term=torch.exp(torch.arange(0,d_model,2)*(-math.log(10000.0)/d_model))

        pe[:,0::2]=torch.sin(position*div_term)
        pe[:,1::2]=torch.cos(position*div_term)

        pe=pe.unsqueeze(1)
        self.register_buffer("pe",pe)

    def forward(self,x):
        return x+self.pe[:,:x.size(0)]

In [ ]:
class attention(nn.Module):
    def __init__(self,num_heads=8,d_model=512):
        super().__init__()

        assert d_model%num_heads==0

        self.d_k=d_model//num_heads
        self.num_heads=num_heads

        self.q_linear=nn.Linear(d_model,d_model)
        self.k_linear=nn.Linear(d_model,d_model)
        self.v_linear=nn.Linear(d_model,d_model)
        self.out=nn.Linear(d_model,d_model)

    def forward(self,q,k,v,mask=None):

        batch_size=q.size(1)

        q=self.q_linear(q).view(batch_size,-1,self.num_heads,self.d_k).transpose(1,2)
        k=self.q_linear(k).view(batch_size,-1,self.num_heads,self.d_k).transpose(1,2)
        v=self.q_linear(v).view(batch_size,-1,self.num_heads,self.d_k).transpose(1,2)

        scores=torch.matmul(q,k.transpose(-2,-1))/math.sqrt(self.d_k)

        if mask is not None:
            scores=torch.masked_fill(mask==0,float("inf"))

        attn=torch.softmax(scores,dim=-1)
        output=torch.matmul(attn,v)
        output=output.transpose(1,2).contiguous().view(batch_size,-1,self.num_heads*self.d_k)

        return self.out(output)


In [ ]:
class feedforward(nn.Module):
    def __init__(self,d_ff=1024,d_model=512,drop=0.2):
        super().__init__()
        self.features=nn.Sequential(
            nn.Linear(d_model,d_ff),nn.ReLU(),nn.Dropout(drop),nn.Linear(d_ff,d_model)
        )
    def forward(self,x):
        return self.features(x)

In [ ]:
class encoder(nn.Module):
    def __init__(self,num_heads=8,d_model=512,d_ff=1024,drop=0.2):
        super().__init__()

        self.attn=attention(num_heads,d_model)
        self.ff=feedforward(d_ff,d_model)
        self.norm1=nn.LayerNorm(d_model)
        self.norm2=nn.LayerNorm(d_model)
        self.drop=nn.Dropout(drop)

    def forward(self,x,mask=None):

        attn=self.attn(x,x,x,mask)
        x=self.norm1(self.drop(attn))
        ff=self.ff(x)
        x=self.norm2(self.drop(ff))

        return x

In [ ]:
class transformer(nn.Module):
    def __init__(self,max_len=100,d_model=512,num_heads=8,d_ff=1024,num_layers=6,drop=0.2,vocab_size=2000):
        super().__init__()

        self.pe=positional_encoding(max_len,d_model)
        encoder_layer=encoder(num_heads,d_model,d_ff,drop)
        self.transformer_encoder=nn.TransformerEncoder(encoder_layer,num_layers)
        self.embedding=nn.Embedding(vocab_size,d_model)
        self.fc_out=nn.Linear(d_model,vocab_size)
        self.drop=nn.Dropout(drop)

    def forward(self,x):
        embed=self.embedding(x)+self.pe[:,:x.size(1),:]
        enc_out=self.transformer_encoder(embed)
        pooled=enc_out.mean(dim=1)
        out=self.fc(self.drop(pooled))
        return out


In [ ]:
model=transformer(vocab_size=vocab_size,max_len=max_len)
model_loss=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min',factor=0.5)

C:\Users\Arun\AppData\Roaming\Python\Python310\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(


In [ ]:
class early_stopping(nn.Module):
    def __init__(self,min_delta,patience):
        super().__init__()

        self.min_delta=min_delta
        self.patience=patience
        self.early_stop=False
        self.best_loss=float("inf")
        self.counter=0

    def __call__(self,val_loss):

        if self.min_delta >= val_loss-self.best_loss:
            self.counter=0
            self.best_loss=val_loss
        else:
            self.counter+=1
            if self.counter>=self.patience:
                self.early_stop=True